[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/AI-HPC-Research-Team/AI-Basics/blob/main/09%20LLM%20%26%20Agent/langchain_agents.ipynb)

![trl banner](https://huggingface.co/datasets/trl-lib/documentation-images/resolve/main/trl_banner_dark.png)

Easily fine-tune Large Language Models (LLMs) or Vision-Language Models (VLMs) with **LoRA** or **QLoRA** using the [**Transformers Reinforcement Learning (TRL)**](https://github.com/huggingface/trl) library built by Hugging Face — all within a **free Google Colab notebook** (powered by a **T4 GPU**.).  

- [TRL GitHub Repository](https://github.com/huggingface/trl) — star us to support the project!  
- [Official TRL Examples](https://huggingface.co/docs/trl/example_overview)  
- [Community Tutorials](https://huggingface.co/docs/trl/community_tutorials)

## Key concepts

- **SFT**: Trains models from example input-output pairs to align behavior with human preferences.
- **LoRA**: Updates only a few low-rank parameters, reducing training cost and memory.
- **QLoRA**: A quantized version of LoRA that enables even larger models to fit on small GPUs.
- **TRL**: The Hugging Face library that makes fine-tuning and reinforcement learning simple and efficient.

Learn how to perform **Supervised Fine-Tuning (SFT)** with **LoRA/QLoRA** using **TRL**.

## Install dependencies

We'll install **TRL** with the **PEFT** extra, which ensures all main dependencies such as **Transformers** and **PEFT** (a package for parameter-efficient fine-tuning, e.g., LoRA/QLoRA) are included. Additionally, we'll install **trackio** to log and monitor our experiments, and **bitsandbytes** to enable quantization of LLMs, reducing memory consumption for both inference and training.

In [ ]:
!pip install -Uq "trl[peft]" trackio bitsandbytes liger-kernel

In [ ]:
import torch, sys, subprocess, os
print("python:", sys.version)
print("torch:", torch.__version__, "CUDA:", torch.version.cuda)
print(subprocess.getoutput("nvidia-smi | head -n 3"))


In [ ]:
# 2) Install vLLM built for cu124 (use official wheels index + backend selector)
!pip install -U uv
!UV_TORCH_BACKEND=cu124 uv pip install "vllm==0.10.2" \
  --extra-index-url https://wheels.vllm.ai/simple


### Log in to Hugging Face

Log in to your **Hugging Face** account to save your fine-tuned model, track your experiment results directly on the Hub or access gated models. You can find your **access token** on your [account settings page](https://huggingface.co/settings/tokens).

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Load model and configure LoRA/QLoRA

This notebook can be used with two fine-tuning methods. By default, it is set up for **QLoRA**, which includes quantization using `BitsAndBytesConfig`. If you prefer to use standard **LoRA** without quantization, simply comment out the `BitsAndBytesConfig` configuration.

Below, choose your **preferred model**. All of the options have been tested on **free Colab instances**.

In [ ]:
from peft import LoraConfig, get_peft_model

# You may need to update `target_modules` depending on the architecture of your chosen model.
# For example, different LLMs might have different attention/projection layer names.
peft_config = LoraConfig(
    r=32,
    lora_alpha=32,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj",],
)

In [ ]:
# Select one model below by uncommenting the line you want to use 👇
## Qwen
# model_id, output_dir = "unsloth/qwen3-14b-unsloth-bnb-4bit", "qwen3-14b-unsloth-bnb-4bit-SFT"     # ⚠️ ~14.1 GB VRAM
# model_id, output_dir = "Qwen/Qwen3-8B", "Qwen3-8B-SFT"                                          # ⚠️ ~12.8 GB VRAM
# model_id, output_dir = "Qwen/Qwen2.5-7B-Instruct", "Qwen2.5-7B-Instruct"                        # ✅ ~10.8 GB VRAM

## Llama
# model_id, output_dir = "meta-llama/Llama-3.2-3B-Instruct", "Llama-3.2-3B-Instruct"              # ✅ ~4.7 GB VRAM
# model_id, output_dir = "meta-llama/Llama-3.1-8B-Instruct", "Llama-3.1-8B-Instruct"              # ⚠️ ~10.9 GB VRAM

## Gemma
# model_id, output_dir = "google/gemma-3n-E2B-it", "gemma-3n-E2B-it"                              # ❌ Upgrade to a higher tier of colab
# model_id, output_dir = "google/gemma-3-4b-it", "gemma-3-4b-it"                                  # ⚠️ ~6.8 GB VRAM

## Granite
# model_id, output_dir = "ibm-granite/granite-4.0-micro", "granite-4.0-micro"                      # ✅ ~3.3 GB VRAM

## LFM2
model_id, output_dir = "LiquidAI/LFM2-2.6B", "LFM2-2.6B-SFT"                                     # ✅ ~5.89 GB VRAM

In [ ]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    attn_implementation="sdpa",                   # Change to Flash Attention if GPU has support
    dtype=torch.float16,                          # Change to bfloat16 if GPU has support
    use_cache=True,                               # Whether to cache attention outputs to speed up inference
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,                        # Load the model in 4-bit precision to save memory
        bnb_4bit_compute_dtype=torch.float16,     # Data type used for internal computations in quantization
        bnb_4bit_use_double_quant=True,           # Use double quantization to improve accuracy
        bnb_4bit_quant_type="nf4"                 # Type of quantization. "nf4" is recommended for recent LLMs
    )
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

## Pre-CPT Evaluation, Inference, and CPT (Continued Pre-Training)
> 1) Pre-CPT evaluation & inference
> 2) CPT (causal LM) + training curves
> 3) Post-CPT evaluation & inference

In [ ]:
import torch
from datasets import load_dataset, DatasetDict
from transformers import DataCollatorForLanguageModeling, TrainingArguments, Trainer

# 1) Load WikiText-2
raw = load_dataset("Salesforce/wikitext", "wikitext-2-raw-v1")["train"]

# 2) Remove empty lines and split into 70/10/20 (train/val/test)
raw = raw.filter(lambda ex: len(ex["text"].strip()) > 0)
tmp = raw.train_test_split(test_size=0.20, seed=42)                 # 20% → test
val_split = tmp["train"].train_test_split(test_size=0.125, seed=42) # 0.125 × 80% = 10% → val

# Sample

def take_n(ds_split, n, seed=42):
    n = int(min(n, len(ds_split)))
    return ds_split.shuffle(seed=seed).select(range(n))

N_TRAIN, N_VAL, N_TEST = 1000, 100, 200

cpt_ds = DatasetDict({
    "train":      take_n(val_split["train"],  N_TRAIN, seed=42),     # 70%
    "validation": take_n(val_split["test"], N_VAL,   seed=42),      # 10%
    "test":       take_n(tmp["test"],N_TEST,  seed=42),           # 20%
})

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# 3) Reuse CPT tokenization / collator / Trainer setup
#    (Assumes `model` and `tokenizer` are already defined from the loaded checkpoint)
block_size = 512
def tok_map_text(batch):
    return tokenizer(batch["text"], truncation=True, max_length=block_size)

tok_train = cpt_ds["train"].map(tok_map_text, batched=True, remove_columns=["text"])
tok_val   = cpt_ds["validation"].map(tok_map_text, batched=True, remove_columns=["text"])

# Causal LM data collator (no masking)
collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
print(cpt_ds)

In [ ]:
# (1) Pre-CPT evaluation & inference
import math, torch
from datasets import Dataset
from torch.utils.data import DataLoader

_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.eval(); model.to(_device)

loader = DataLoader(tok_val, batch_size=2, shuffle=False, collate_fn=collator)
tot_loss, tot_tokens = 0.0, 0
with torch.no_grad():
    for batch in loader:
        batch = {k:v.to(_device) for k,v in batch.items()}
        out = model(**batch)
        loss = out.loss
        pad = tokenizer.pad_token_id
        token_count = (batch["labels"]!=pad).sum().item() if pad is not None else batch["labels"].numel()
        tot_loss += loss.item()*token_count
        tot_tokens += token_count

avg_nll = tot_loss/max(1,tot_tokens)
ppl = math.exp(avg_nll) if avg_nll<30 else float("inf")
print(f"[Pre-CPT] token-avg NLL={avg_nll:.4f} | PPL={ppl:.2f}")

# (2) Single inference
messages = [
    {"role": "system", "content": "You are a helpful assistant. Answer in English only."},
    {"role": "user", "content": "Summarize CPT in one sentence."},
]
text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(text, return_tensors="pt").to(next(model.parameters()).device)
gen = model.generate(**inputs, max_new_tokens=80, do_sample=True, temperature=0.7, top_p=0.9)
print(">>> PROMPT:\n", text)
print("\n>>> GENERATION:\n", tokenizer.decode(gen[0], skip_special_tokens=True))

### ▶ CPT: Continued Pre-Training (causal LM)

In [ ]:
# (2) CPT training using transformers.Trainer (causal LM, mlm=False)
import torch
from datasets import Dataset
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling


# Minimal training args for classroom/Colab demos; adjust as needed
args_cpt = TrainingArguments(
    output_dir="cpt-output",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    learning_rate=5e-5,
    warmup_ratio=0.03,
    logging_steps=5,
    eval_strategy="steps",
    eval_steps=20,
    save_strategy="no",
    report_to=[],
    bf16=torch.cuda.is_available(),  # use bf16 on GPUs that support it; otherwise ignore
)

trainer = Trainer(
    model=model,  # your loaded checkpoint (works with PEFT/LoRA-wrapped models as well)
    args=args_cpt,
    train_dataset=tok_train,
    eval_dataset=tok_val,
    data_collator=collator,
)

train_res = trainer.train()
print(train_res)
trainer.save_model("cpt-output"); tokenizer.save_pretrained("cpt-output")
# Plot training curves
import matplotlib.pyplot as plt

logs = trainer.state.log_history
steps, tr_loss, ev_steps, ev_loss = [], [], [], []
for rec in logs:
    if "loss" in rec and "learning_rate" in rec:
        steps.append(rec.get("step", len(steps)))
        tr_loss.append(rec["loss"])
    if "eval_loss" in rec:
        ev_steps.append(rec.get("step", len(ev_steps)))
        ev_loss.append(rec["eval_loss"])

plt.figure(figsize=(6,4))
if tr_loss: plt.plot(steps, tr_loss, label="train/loss")
if ev_loss: plt.plot(ev_steps, ev_loss, label="eval/loss")
plt.xlabel("step"); plt.ylabel("loss"); plt.title("CPT curves"); plt.legend(); plt.tight_layout(); plt.show()

In [ ]:

# (3) Post-CPT evaluation & inference
import math, torch
from transformers import AutoModelForCausalLM
_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_cpt = AutoModelForCausalLM.from_pretrained("cpt-output",
    attn_implementation="sdpa",                   # Change to Flash Attention if GPU has support
    dtype=torch.float16,                          # Change to bfloat16 if GPU has support
    use_cache=True,                               # Whether to cache attention outputs to speed up inference
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,                        # Load the model in 4-bit precision to save memory
        bnb_4bit_compute_dtype=torch.float16,     # Data type used for internal computations in quantization
        bnb_4bit_use_double_quant=True,           # Use double quantization to improve accuracy
        bnb_4bit_quant_type="nf4"                 # Type of quantization. "nf4" is recommended for recent LLMs
    )
    ).to(_device)
model_cpt.eval()

tot_loss, tot_tokens = 0.0, 0
with torch.no_grad():
    for batch in loader:
        batch = {k:v.to(_device) for k,v in batch.items()}
        out = model_cpt(**batch); loss = out.loss
        pad = tokenizer.pad_token_id
        token_count = (batch["labels"]!=pad).sum().item() if pad is not None else batch["labels"].numel()
        tot_loss += loss.item()*token_count
        tot_tokens += token_count

avg_nll = tot_loss/max(1,tot_tokens)
ppl = math.exp(avg_nll) if avg_nll<30 else float("inf")
print(f"[Post-CPT] token-avg NLL={avg_nll:.4f} | PPL={ppl:.2f}")

In [ ]:
# (3) Inference after CPT
from transformers import AutoModelForCausalLM

messages = [
    {"role": "system", "content": "You are a helpful assistant. Answer in English only."},
    {"role": "user", "content": "Summarize CPT in one sentence."},
]
text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(text, return_tensors="pt").to(next(model_cpt.parameters()).device)
gen = model_cpt.generate(**inputs, max_new_tokens=80, do_sample=True, temperature=0.7, top_p=0.9)
print(">>> PROMPT (post-CPT):\n", text)
print("\n>>> GENERATION (post-CPT):\n", tokenizer.decode(gen[0], skip_special_tokens=True))

## Load Dataset

In this step, we load the [**HuggingFaceH4/Multilingual-Thinking**](https://huggingface.co/datasets/HuggingFaceH4/Multilingual-Thinking) dataset from the Hugging Face Hub using the `datasets` library.  
This dataset focuses on **multilingual reasoning**, where the *chain of thought* has been translated into several languages such as French, Spanish, and German.  
By fine-tuning a reasoning-capable model on this dataset, it learns to **generate reasoning steps in multiple languages**, making its thought process more **interpretable and accessible** to non-English speakers.

> 💡 This dataset is best suited for models that already demonstrate reasoning capabilities.  
> If you're using a model without reasoning skills, consider choosing a different dataset. Example: [`trl-lib/llava-instruct-mix`](https://huggingface.co/datasets/trl-lib/llava-instruct-mix).

For efficiency, we'll load only the **training split**:

In [ ]:
from datasets import load_dataset

dataset_name = "HuggingFaceH4/Multilingual-Thinking"
train_dataset = load_dataset(dataset_name, split="train")

This dataset contains different columns. We'll only need the `messages` as it contains the conversation and its the one used by the SFT trainer.

In [ ]:
train_dataset

Let's see a full example to understand the internal structure:

In [ ]:
train_dataset[0]


Now, let's remove the columns that are not needed, as we just discussed:

In [ ]:
train_dataset = train_dataset.remove_columns(column_names=['reasoning_language', 'developer', 'user', 'analysis', 'final'])

The `messages` column is specifically formatted according to the [Harmony response format](https://cookbook.openai.com/articles/openai-harmony) used by *gpt-oss*.  
In our case, we'll need to simplify it slightly, since our model's chat template doesn't include a dedicated `thinking` section (check [this example](https://cookbook.openai.com/articles/gpt-oss/fine-tune-transfomers) for more details).  
To adapt it, we'll merge that part into the message content using the standard `<think>...</think>` tags.


In [ ]:
def merge_thinking_and_remove_key(example):
    new_messages = []
    for msg in example["messages"]:
        content = msg["content"]
        thinking = msg.pop("thinking", None)
        if thinking and isinstance(thinking, str) and thinking.strip():
            content = f"<think>\n{thinking}\n</think>\n{content}"
        msg["content"] = content
        new_messages.append(msg)
    example["messages"] = new_messages
    return example

train_dataset = train_dataset.map(merge_thinking_and_remove_key)

Let's load the selected model using `transformers`, configuring QLoRA via `bitsandbytes` (you can remove it if doing LoRA). We don't need to configure the tokenizer since the trainer takes care of that automatically.

## Train model

We'll configure **SFT** using `SFTConfig`, keeping the parameters minimal so the training fits on a free Colab instance. You can adjust these settings if more resources are available. For full details on all available parameters, check the [TRL SFTConfig documentation](https://huggingface.co/docs/trl/sft_trainer#trl.SFTConfig).

The following cell defines LoRA (or QLoRA if needed). When training with LoRA/QLoRA, we use a **base model** (the one selected above) and, instead of modifying its original weights, we fine-tune a **LoRA adapter** — a lightweight layer that enables efficient and memory-friendly training. The **`target_modules`** specify which parts of the model (e.g., attention or projection layers) will be adapted by LoRA during fine-tuning.

In [ ]:
from trl import SFTConfig

training_args = SFTConfig(
    # Training schedule / optimization
    per_device_train_batch_size = 1,      # Batch size per GPU
    gradient_accumulation_steps = 4,      # Gradients are accumulated over multiple steps → effective batch size = 2 * 8 = 16
    warmup_steps = 5,
    # num_train_epochs = 1,               # Number of full dataset passes. For shorter training, use `max_steps` instead (this case)
    max_steps = 30,
    learning_rate = 2e-4,                 # Learning rate for the optimizer
    optim = "paged_adamw_8bit",           # Optimizer

    # Logging / reporting
    logging_steps=1,                      # Log training metrics every N steps
    report_to="trackio",                  # Experiment tracking tool
    trackio_space_id=output_dir,          # HF Space where the experiment tracking will be saved
    output_dir=output_dir,                # Where to save model checkpoints and logs

    max_length=1024,                      # Maximum input sequence length
    use_liger_kernel=True,                # Enable Liger kernel optimizations for faster training
    activation_offloading=True,           # Offload activations to CPU to reduce GPU memory usage
    gradient_checkpointing=True,          # Save memory by re-computing activations during backpropagation

    # Hub integration
    push_to_hub=True,                     # Automatically push the trained model to the Hugging Face Hub
                                          # The model will be saved under your Hub account in the repository named `output_dir`

    gradient_checkpointing_kwargs={"use_reentrant": False}, # To prevent warning message
)

Configure the SFT Trainer. We pass the previously configured `training_args`. We don't use eval dataset to mantain memory usage low but you can configure it.

In [ ]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model_cpt,
    args=training_args,
    train_dataset=train_dataset,
    peft_config=peft_config
)

Show memory stats before training

In [ ]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)

print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

And train!

In [ ]:
trainer_stats = trainer.train()

Show memory stats after training

In [ ]:
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)

print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

The training procedure generates both standard training logs and **trackio** logs, which help us monitor the training progress. Example outputs would look like the following:

![sft-lora-notebook-trackio](https://huggingface.co/datasets/trl-lib/documentation-images/resolve/main/sft-lora-notebook-trackio.png)

## Saving fine tuned model

In this step, we save the fine-tuned model both **locally** and to the **Hugging Face Hub** using the credentials from your account.

In [ ]:
trainer.save_model(output_dir)
trainer.push_to_hub(dataset_name=dataset_name)

## Load the fine-tuned model and run inference

Now, let's test our fine-tuned model by loading the **LoRA/QLoRA adapter** and performing **inference**. We'll start by loading the **base model**, then attach the adapter to it, creating the final fine-tuned model ready for evaluation.

In [ ]:
from getpass import getpass

your_HF_username = getpass("Enter your HF username: ")

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

adapter_model = f"{your_HF_username}/{output_dir}"

base_model = AutoModelForCausalLM.from_pretrained(model_id, dtype="auto", device_map="auto")

tokenizer = AutoTokenizer.from_pretrained(model_id)

Let's create a sample message using the dataset's structure. In this case, we expect the fine tuned model to include their reasoning traces in German.

In [ ]:
messages = [
  {
      'content': 'reasoning language: German\n\nAlways refuse to answer, responding simply \'No\'',
      'role': 'system',
  },
  {
      'content': "Can you check how many followers I currently have on my Twitter account?",
      'role': 'user',
  }
]

Let's first check what's the output for the base model, without the adapter.

In [ ]:
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(base_model.device)

generated_ids = base_model.generate(
    **model_inputs,
    max_new_tokens=512
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):]

# Decode and extract model response
generated_text = tokenizer.decode(output_ids, skip_special_tokens=True)
print(generated_text)

We can see that the reasoning traces are in English, which is expected. Let's now load the fine-tuned model and check its answer.

In [ ]:
fine_tuned_model = PeftModel.from_pretrained(base_model, adapter_model)

In [ ]:
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(fine_tuned_model.device)

generated_ids = fine_tuned_model.generate(
    **model_inputs,
    max_new_tokens=512
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):]

# Decode and extract model response
generated_text = tokenizer.decode(output_ids, skip_special_tokens=True)
print(generated_text)

The model now generates its reasoning trace in German!

## (Optional) Inference and Serving with vLLM

You can use Transformer models with **vLLM** to serve them in real-world applications. Learn more [here](https://blog.vllm.ai/2025/04/11/transformers-backend.html).

Push Merged Model (for LoRA or QLoRA Training)

To serve the model via **vLLM**, the repository must contain the merged model (base model + LoRA adapter). Therefore, you need to upload it first.

In [ ]:
# may cost more than 3 minutes
model_merged = fine_tuned_model.merge_and_unload()

save_dir = f"{output_dir}-merged"

model_merged.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

In [ ]:
# may cost more than 5 minutes
model_merged.push_to_hub(f"{your_HF_username}/{output_dir}-merged")
tokenizer.push_to_hub(f"{your_HF_username}/{output_dir}-merged")

### Performing Inference with vLLM

Use **vLLM** to run your model and generate text efficiently in real-time. This allows you to test and deploy your fine-tuned models with low latency and high throughput.

In [ ]:
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer
import torch
import os
os.environ["VLLM_LOGGING_LEVEL"] = "DEBUG"
your_HF_username="jinraolan"
output_dir="LFM2-2.6B-SFT"
llm = LLM(
    model=f"{your_HF_username}/{output_dir}-merged",
    trust_remote_code=True,
    dtype="float16",
    max_model_len=256,
    tensor_parallel_size=1,
    gpu_memory_utilization=0.85,
    enforce_eager=True

)

In [ ]:
from vllm import LLM, SamplingParams
import os
os.environ["VLLM_LOGGING_LEVEL"] = "DEBUG"
your_HF_username="jinraolan"
output_dir="LFM2-2.6B-SFT"
llm = LLM(f"{your_HF_username}/{output_dir}-merged",
          trust_remote_code=True, dtype="float16",
          max_num_batched_tokens=1,
          max_model_len=256, gpu_memory_utilization=0.95,
          enforce_eager=True)


In [ ]:
# need mprm GPU RAM
outs = llm.generate(prompts=["Answer in English only: Summarize CPT in one sentence."],
                sampling_params=SamplingParams(max_tokens=16))
print(outs[0].outputs[0].text)